In [51]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

In [52]:
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [48]:
movie_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 영화 전문가입니다.",
        ),
        FewShotChatMessagePromptTemplate(
            example_prompt=ChatPromptTemplate.from_messages(
                [
                    ("human", "영화 {movie}에 대해서 알아?"),
                    ("ai", "{answer}"),
                ]
            ),
            examples=[
                {
                    "movie": "다크 나이트",
                    "answer": "감독: 크리스토퍼 놀란\n"
                    "예산: $185,000,000\n"
                    "흥행 수익: $1,008,486,720\n"
                    "영화의 장르: 액션, 시네마, 드라마\n"
                    "간단한 시놉시스: 각종 범죄가 난무하는 고담시를 지키는 배트맨, 어느 날 도시에 광기 어린 악당 조커가 등장해 사람들을 두려움에 빠뜨린다. 조커는 배트맨이 정체를 밝힐 때까지 사람들을 죽이겠다고 협박하고, 배트맨은 자신의 모든 것을 걸고 최악의 악당 조커와 맞서기 시작한다.\n",
                },
                {
                    "movie": "다크 나이트 라이즈",
                    "answer": "감독: 크리스토퍼 놀란\n"
                    "주요 출연진: 크리스찬 베일, 앤 해서웨이, 톰 하디\n"
                    "예산: $250,000,000\n"
                    "흥행 수익: $1,081,041,287\n"
                    "영화의 장르: 액션, 시네마, 드라마\n"
                    "간단한 시놉시스: 배트맨이 8년 동안 사라진 후 고담시는 평화롭지만 위험한 상황에 처하게 된다. 새로운 악당 베인이 등장하여 도시를 혼란에 빠뜨리고, 배트맨은 다시 나타나서 베인과의 대결에 나선다. 배트맨은 자신의 한계를 극복하고 도시를 구하기 위해 힘을 모으는데...\n",
                },
                {
                    "movie": "괴물",
                    "answer": "감독: 봉준호\n"
                    "주요 출연진: 송강호, 변희봉, 박해일, 배두나, 고아성\n"
                    "예산: 110억 원\n"
                    "흥행 수익: $89,433,436\n"
                    "영화의 장르: 드라마, 호러, SF\n"
                    "간단한 시놉시스: 아버지(변희봉)가 운영하는 한강매점, 늘어지게 낮잠 자던 강두(송강호)는 우연히 특이한 광경을 목격하게 된다. 생전 보도 못 한 무언가가 한강다리에 매달려 움직이는 것이다. 정체를 알 수 없는 괴물은 둔치 위로 올라와 사람들을 거침없이 깔아뭉개고, 무차별로 물어뜯기 시작한다. 순식간에 아수라장으로 돌변하는 한강변. 강두도 뒤늦게 딸 현서(고아성)를 데리고 정신없이 도망가지만, 꼭 잡았던 현서의 손을 놓치고 만다. 하루아침에 집과 생계, 그리고 현서까지 모든 것을 잃게 된 강두 가족. 돈도 없고 빽도 없는 그들은 위험 구역으로 선포된 한강 어딘가에 있을 현서를 찾아 나선다.\n",
                },
            ],
        ),
        ("human", "영화 {movie}에 대해서 알아?"),
    ]
)

chain = movie_prompt | chat

In [49]:
chain.invoke({"movie": "어벤져스"})

감독: 조스 웨던
주요 출연진: 로버트 다우니 주니어, 크리스 에반스, 크리스 헴스워스, 스칼릿 요한슨, 마크 러팔로
예산: $220,000,000
흥행 수익: $1,518,812,988
영화의 장르: 액션, 모험, SF
간단한 시놉시스: 지구를 위협하는 강력한 적인 로키가 지구로 내려오고, 세계평화를 위해 모인 히어로들인 아이언맨, 캡틴 아메리카, 토르, 헐크, 블랙 위도우, 호크아이가 모여 어벤져스를 결성하여 로키와의 전투에 나선다. 다양한 히어로들이 힘을 합쳐 세계를 구하는 과정을 다룬 영화이다.

AIMessageChunk(content='감독: 조스 웨던\n주요 출연진: 로버트 다우니 주니어, 크리스 에반스, 크리스 헴스워스, 스칼릿 요한슨, 마크 러팔로\n예산: $220,000,000\n흥행 수익: $1,518,812,988\n영화의 장르: 액션, 모험, SF\n간단한 시놉시스: 지구를 위협하는 강력한 적인 로키가 지구로 내려오고, 세계평화를 위해 모인 히어로들인 아이언맨, 캡틴 아메리카, 토르, 헐크, 블랙 위도우, 호크아이가 모여 어벤져스를 결성하여 로키와의 전투에 나선다. 다양한 히어로들이 힘을 합쳐 세계를 구하는 과정을 다룬 영화이다.')

In [50]:
chain.invoke({"movie": "기생충"})


감독: 봉준호
주요 출연진: 송강호, 이선균, 조여정, 최우식
예산: 약 110억 원
흥행 수익: $258,817,045
영화의 장르: 드라마, 스릴러, 흑색 코미디
간단한 시놉시스: 기생충은 한 가족이 빈부격차를 이용하여 다른 가정에 기생하면서 벌어지는 사건을 그린 영화입니다. 한 가족은 부자 가정에 각각 다른 신분으로 취업하여 들어가며 서서히 가정 내부에 침투하게 됩니다. 그러나 예상치 못한 사건들이 벌어지면서 이야기는 더욱 복잡해지고 긴장감을 높이게 됩니다. 영화는 사회적 격차와 가족의 관계, 인간 본성에 대한 다층적인 메시지를 담고 있습니다.

AIMessageChunk(content='감독: 봉준호\n주요 출연진: 송강호, 이선균, 조여정, 최우식\n예산: 약 110억 원\n흥행 수익: $258,817,045\n영화의 장르: 드라마, 스릴러, 흑색 코미디\n간단한 시놉시스: 기생충은 한 가족이 빈부격차를 이용하여 다른 가정에 기생하면서 벌어지는 사건을 그린 영화입니다. 한 가족은 부자 가정에 각각 다른 신분으로 취업하여 들어가며 서서히 가정 내부에 침투하게 됩니다. 그러나 예상치 못한 사건들이 벌어지면서 이야기는 더욱 복잡해지고 긴장감을 높이게 됩니다. 영화는 사회적 격차와 가족의 관계, 인간 본성에 대한 다층적인 메시지를 담고 있습니다.')